### Algorithm
- Suppose we have a (1000,2) size data
1. #### Preprocessing
    - Find a vector, for projecting the data points on it

2. #### Covariance matrix
    - For 2 features, we get a 2 x 2 matrix
$$Cov(x,y) = \sum\limits_{i=1}^m\dfrac{(x - \bar{x})(y - \bar{y})}{N-1}$$
Hence are matrix looks like,
$$ \begin{bmatrix} Cov(x_1, x_1) & Cov(x_1, x_2) \\
Cov(x_2, x_1) & Cov(x_2, x_2) \end{bmatrix} $$
Which turns out to be a symmetric matrix
  

3. #### Eigen vector calculation
    - Find the eigen vector the get the direction of the vector for projecting
    - We use a function called `SVD` (Singular Value Decompositions)
    - SVD gives us U, S, V
    - We only use U
    - U is a $n \times n$ matrix, and we choose the best k
    - `U = U[:, :k]` $(n \times k)$

4. #### Projection
    - $Z^{(i)} = U_{red}^T \times x^{(i)} $
    - To get Z of $(k \times 1)$ shape